In [1]:
import sys,importlib
sys.path.append('../')
import simulation
from numpy import *
import plotly.offline as py
import plotly.graph_objs as go


### sub-system

In [2]:
alpha = 1
delta = .25
gamma = .3
beta = -1
omega = 1
def dufffingOscillator(state,t):
    x,y = state[0],state[1]
    xdot = y
    ydot = -alpha*(x**3) - beta* x - delta*y + gamma*cos(omega*t)
    return asarray((xdot,ydot))

### coupling

In [3]:
a0,a1 = .5,.2
w = 1.3


In [7]:
def coupling1(stateA,stateB,t):
    x1,y1 = stateA
    x2,y2 = stateB
    xdot = y2/(a0+a1*sin(w*t))
    ydot = 0
    return asarray((xdot,ydot))

In [8]:
def coupling2(stateA,stateB,t):
    x1,y1 = stateA
    x2,y2 = stateB
    xdot = y1*(a0+a1*sin(w*t))+a1*x1*w*cos(w*t)
    ydot = 0
    return asarray((xdot,ydot))

In [9]:
def stabilizer(stateA,stateB,t):
    x1,y1 = stateA
    x2,y2 = stateB
    xdot = -(x2-x1*(a0 + a1*sin(w*t)))
    ydot = 0
    return asarray((xdot,ydot))

### simulation

In [10]:
time = 100
delTime = .1
tail = 0
timeline = arange(0.0, time, delTime)

In [11]:
stateA0 = asarray([0.15,0.25])
stateB0 = asarray([1.5,0.34])
stateA,stateB = simulation.evolveAB(dufffingOscillator,dufffingOscillator,stateA0,stateB0,couplingA=coupling1,couplingB=coupling2,stabilizerB=stabilizer,time=time,delTime=delTime,tail=tail)

### synchronization - order parameter

In [12]:
py.iplot([go.Scatter(x=timeline,y=stateB[:,0]/stateA[:,0])])

### eigenvalues

In [15]:
def eigenvalue(stateA,stateB,t):
   
    return -((a0+a1*sin(w*t))**2)-1

In [16]:
py.iplot([go.Scatter(x=timeline,y=eigenvalue(stateA,stateB,timeline))])